# Python 활용한 데이터 전처리/시각화
# Chapter 1. 데이터 전처리

데이터 분석/AI 모델링을 위하여 데이터를 정제하는 전처리 방법을 실습해 봅시다.    
우선 확보한 데이터를 전처리를 위해 불러옵니다.     

In [ ]:
# 판다스 불러오기
import pandas as pd

In [ ]:
from impala.dbapi import connect
import pandas as pd

# Step 1: Impala 연결 설정
conn = connect(
    host='apramn102.hyundaicapital.com',  # Hue에서 사용하는 Impala 호스트 주소
    port=21053,               # Impala 기본 포트
    auth_mechanism='GSSAPI'   # 사용할 데이터베이스 이름
)

# Step 2: SQL 쿼리 실행 및 DataFrame으로 변환
query = "SELECT * FROM hcs_t_bdpedu_l2a.sc_cust;"
df = pd.read_sql(query, conn)

# Step 3: 데이터프레임 확인
print(df)

# Step 4: 연결 닫기
conn.close()

In [ ]:
print(df)

## 1. 데이터 탐색하기
### <b>데이터 확인하기</b>
Dataframe에서 제공하는 메소드를 활용하여 데이터의 구성을 확인합니다.    
info는 데이터 구성과 특성을 확인해 볼 수 있습니다

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.describe()

### <b>필요한 컬럼만 추출하기</b>
수집한 데이터의 컬럼이 너무 많을 경우 데이터 처리에 불필요한 시간과 자원이 계속 소모 되기에,     
이번 수업에서는 필요한 컬럼만 추출하여 데이터처리를 하도록 하겠습니다.    
실제 분석/모델링에서는 어떤 컬럼이 중요할지 알수 없기 떄문에 자원이 가능한 많은 데이터를 가지고 하셔도 좋습니다.

In [ ]:
#데이터 처리에 필요한 10개 항목만 추출
#고객등급(cust_class), 성별(sex_type), 나이(age), 사용서비스수(efct_svc_count), 서비스중지여부 (dt_stop_yn), 미납여부(npay_yn)
#3개월 평균 요금(r3m_avg_bill_amt), A서비스 3개월 평균요금(r3m_A_avg_arpu_amt), B서비스 3개월 평균요금(r3m_B_avg_arpu_amt), 해지여부(termination_yn)
cust=df[["cust_class","sex_type","age","efct_svc_count","dt_stop_yn","npay_yn","r3m_avg_bill_amt","r3m_A_avg_arpu_amt","r3m_B_avg_arpu_amt", "termination_yn"]]
cust.head()

실습의 편의를 위해 컬럼명을 조금 더 간결하고 직관적으로 변경해 보겠습니다.     
실제 여러명이 동시에 진행하는 분석 프로젝트나 모델링에서는 정해진 네이밍 룰에 따라 변경하거나 원천 컬럼명을 그대로 사용하는 것이 좋습니다.

In [ ]:
# 컬럼명을 간결하고 직관적으로 변경해 줍시다.
cust=cust.rename(columns = {"cust_class" : 'class',"sex_type":'sex', "efct_svc_count":'service', "dt_stop_yn":'stop',"npay_yn":'npay', "r3m_avg_bill_amt":'avg_bill', "r3m_A_avg_arpu_amt":"A_bill", "r3m_B_avg_arpu_amt":'B_bill', "termination_yn":'termination'})
cust.head()

In [ ]:
cust.info()

## 2. 결측치 처리하기
### <span style="color:RED">주의!!</span> 데이터 결측치 처리시 반드시 원본 데이터를 COPY 해서 사용할것!!
파이썬에서 Copy 메소드를 사용하지 않으면 주소값을 복사해서 사용기 때문에 원본 값을 변경 시키게 됩니다.
따라서 원본 항목을 보전하면서 데이터를 보정하려면 copy 메소드를 사용 해주셔야 합니다.

### <b>결측치 채우기 </b>

다음은 사용자가 지정하는 단일값으로 채워서 결측치를 처리하는 방법입니다.

In [ ]:
cust.head()

In [ ]:
#fillna 함수를 사용해서 특정 숫자나 문자로 결측치를 처리하는 방법
cust=cust.fillna(15)
cust.head()

In [ ]:
cust.info()

fillna의 Method 파라미터를 사용하면,     
사용자가 지정하는 일괄적인 값이 아닌 주변값을 활용하여 결측치를 채울수 있습니다.
주의! Method 파리미터사용시 첫 Record 또는 마지막 Record가 결측치 인지 확인해야 합니다.  

In [ ]:
# 뒤에 있는 data를 사용해서 결측치를 처리하는 방법
cust=cust_fix.copy()
cust=cust.fillna(method='bfill')
cust.head()

In [ ]:
# 앞에 있는 data를 사용해서 결측치를 처리하는 방법
cust=cust_fix.copy()
cust=cust.fillna(method='ffill')
cust.head()

fillna 내 기술 통계량을 사용하면,
사용자가 원하는 값(중위값, 평균값, 최대값, 최솟값)으로 채울 수 있습니다

In [ ]:
# 기술통계량을 사용해서 결측치를 처리하는 방법
# 원본을 변경하고 싶을 때는 inplace = True를 활용
cust=cust_fix.copy()
cust.fillna(df['age'].mean(), inplace=False)
cust.head()

In [ ]:
# 문제
# fillna 메서드를 활용하여 age 컬럼의 결측치를 중위값(median)으로 채우시오
cust=cust_fix.copy()




### <b>결측치 제거하기</b>
결측치를 채우기로 처리하다보면 경우에 따라 값의 왜곡이 크게 발생하는 발생 하는 경우가 있습니다.    
이때는 제거하기 기법을 사용하여 결측치를 처리하면 데이터의 정합성을 보존 할 수 있습니다.

* listwise 방식 : record의 항목 중 1개의 값이라도 NA이면 해당 데이터 행 전체 제거
* pairwise 방식 : 모든 항목이 NA인 데이터 행만 제거

In [ ]:
#listwise 방식으로 제거 하기
#listwise 방식이 디폴트
# axis = 0 이 디폴트
cust=cust_fix.copy()
cust=cust.dropna(how='any')
cust.info()

In [ ]:
#pairwise 방식으로 제거하기
cust=cust_fix.copy()
cust=cust.dropna(how='all')
cust.info()

NA의 갯수에 따라 결측데이터를 판단하고자 한다면, thresh 파라미터를 사용하면 됩니다.    
NA가 아닌값이 n개 이상인 경우만 남겨라는 뜻으로 해석하면 됩니다.

In [ ]:
#임계치를 설정해서 제거하기
cust=cust_fix.copy()
cust=cust.dropna(thresh=10)
cust.info()

In [ ]:
# 특정열 안에서만 삭제하기|
cust=cust_fix.copy()
cust=cust.dropna(subset=['age'])
cust.info()

특정 열에 있는 NA만 참고하여 결측치를 제거하려면 Subset 파라미터를 사용하면 됩니다.

### <b>데이터 타입 변경하기</b>
데이터를 불러온 후에는 반드시 데이터 타입을 확인해 주시는것이 좋습니다.    
숫자형 데이터가 문자형으로 지정되어 있거나 혹은 그반의 경우 원하는 데이터 처리 결과가 도출 되지 않을 수 있습니다.

In [ ]:
cust.dtypes

In [ ]:
#age 항목의 type 변경
cust=cust.astype({'age': int })
#cust['age'] = cust['age'].astype(int)

In [ ]:
cust.info()

In [ ]:
cust.head()

## 3. 이상치 처리하기
### <b>범주형 데이터 이상치 처리 하기</b>
범주형 데이터의 경우 value_counts 메소드를 사용하면, 값의 분포와 함께 Trash 값이 있는지도 확인 가능합니다.

In [ ]:
#범주형 데아터의 값 분포 확인하기
print(cust['sex'].value_counts())
print(cust['class'].value_counts())
print(cust['npay'].value_counts())

Class의 값에는 'C, D, E, F, G, H'가 들어있는것을 확인할수 있습니다.    
실제로 그렇지는 않지만 'H'가 실제로는 존재하지 않는 값이라고 가정해 봅시다.    
이상치를 처리하는 방법은 제거하기와 변경하기가 있습니다.

In [ ]:
#이상치 제거하기
cust_data=cust[(cust['class']!='H')]
print(cust_data['class'].value_counts())

In [ ]:
# 문제
# cust의 age 컬럼 내 75세 초과는 이상치로 규정, 75세 초과 삭제
# 이상치가 제거된 버전의 데이터프레임명은 cust_data2


In [ ]:
#이상치 변경하기
cust_data=cust.copy()
cust_data['class']=cust_data['class'].replace('H','F')
print(cust_data['class'].value_counts())

### <b>수치형 데이터 이상치 처리 하기</b>
describe 함수를 사용해서 수치형 변수의 기술 통계를 확인 할 수 있습니다.

In [ ]:
cust.describe()

Q1와 Q3가 거리의 1.5배가 넘어가는 값을 Outlier 라고 합니다.    
이 값들은 이상치로써 일반적으로 제거 또는 변경하여 데이터를 분석,학습 합니다.    
단, 이상치 분석시에는 제거하지 않습니다.    

In [ ]:
#이상치를 제거하는 함수 만들기
def removeOutliers(x, column):
    # Q1, Q3구하기
    q1 = x[column].quantile(0.25)
    q3 = x[column].quantile(0.75)

    # 1.5 * IQR(Q3 - Q1)
    iqr = 1.5 * (q3 - q1)

    # 이상치를 제거
    y=x[(x[column] < (q3 + iqr)) & (x[column] > (q1 - iqr))]

    return(y)

In [ ]:
#연속형 데이터의 이상치 제거하기
cust_data=removeOutliers(cust, 'avg_bill')
cust_data.describe()

In [ ]:
cust_data.info()

In [ ]:
# 동일한 함수로 나머지 연속형 변수에 대해서도 이상치를 처리
cust_data=removeOutliers(cust_data, 'A_bill')
cust_data=removeOutliers(cust_data, 'B_bill')
cust_data.describe()

In [ ]:
cust_data.info()

<br><br>

## 4. Feature Engineering
초기 데이터로부터 특징을 가공하고 생산하여 입력 데이터를 생성하는 과정을 Feature Engineering이라고 합니다.
Feature Engineering을 통해 AI가 학습하기 좋은 형태로 데이터를 만들거나,
새로운 입력 데이터를 생성 할수 있습니다.
이때 데이터를 수정하지 않도록 주의 합니다.

In [ ]:
cust_data.head()

### <b>Binning</b>
Binning은 단어 뜻 그대로 자료를 일정한 규격의 통에 넣는것 입니다.
만약 관측치가 연속형이면서 범위가 너무 다양할 경우, 적절히 그룹을 지어주면 데이터를 이해하기가 더 쉬워질수 있기에 사용합니다.
즉, 연속형 변수를 범주형 변수로 만드는 방법이라고 보시면 됩니다.

In [ ]:
# age를 활용하여 나이대("by_age")  Feature 만들기
cust_data['by_age']=cust_data['age']//10*10
cust_data=cust_data.astype({'age': int, 'by_age':int})

In [ ]:
cust_data.head()

#### cut
Cut 함수를 사용하면 구간을 지정해서 쉽게 범주화 할 수 있습니다.    
Bins 구간 사이의 값을 범주화 하여 Label에 지정된 카테고리명을 사용합니다.



In [ ]:
q1=cust_data['avg_bill'].quantile(0.25)
q3=cust_data['avg_bill'].quantile(0.75)
print(q1,q3)

In [ ]:
#cut 메소드를 활용하여 요금을 3개 구간으로 나누기
cust_data['bill_rating'] = pd.cut(cust_data["avg_bill"], bins=[0,q1,q3,cust_data["avg_bill"].max()] , labels=['low', 'mid','high'])
cust_data.head()

In [ ]:
print(cust_data['bill_rating'].value_counts())

#### qcut
cut비슷하지만 같은 크기로 구간을 나누어 범주형 변수로 만듭니다

In [ ]:
#qcut 메소드를 활용하여 요금을 동일 비율로 3개 구간으로 나누기
cust_data['bill_rating'] = pd.qcut(cust_data["avg_bill"], 3 , labels=['low', 'mid','high'])
cust_data.head()

In [ ]:
print(cust_data['bill_rating'].value_counts())

### <b>Scaling
각 컬럼에 들어있는 데이터의 상대적 크기에 따라 분석 결과나 모델링 결과가 달라질수 있습니다.    
0 ~ 1000까지의 값을 가지는 변수 A와 0 ~ 10까지의 값을 가지는 변수 B를 가지고 분석을 수행하면 상대적으로 큰 숫자를 가지는 A변수의 영향이 더 크게 반영 됩니다.    
따라서, 숫자데이터는 상대적 크기 차이를 제거할 필요가 있고, 이를 scaling이라고 합니다.    

#### Standardization
정규 분포를 평균이 0 이고 분산이 1 인  표준 정규 분포로 변환합니다.

In [ ]:
cust_data_num = cust_data[['avg_bill', 'A_bill', 'B_bill']]
#표준화
Standardization_df = (cust_data_num - cust_data_num.mean())/cust_data_num.std()
Standardization_df.head()

In [ ]:
Standardization_df.describe()

####  MinMax Scaler (nomalization)
값들을 모두 0과 1사이의 값으로 변환합니다.

In [ ]:
#사이킷런 패키지의 MinMaxScaler를 이용하여  Scaling 하기
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()

normalization_df=cust_data_num.copy()
normalization_df[:]=scaler.fit_transform(normalization_df[:])
normalization_df.head()

In [ ]:
normalization_df.describe()

In [ ]:
#MinMaxScaler 함수 구현하기
normalization_df = (cust_data_num - cust_data_num.min())/(cust_data_num.max()-cust_data_num.min())
normalization_df.head()

### <b>One-Hot Encording

원-핫 인코딩은 간단히 말해 한 개의 요소는 True, 나머지 요소는 False로 만들어 주는 기법입니다.    
기계 학습의 주요 문제 중 하나는 많은 알고리즘이 범주 형 데이터를 입력값으로 수용하지 않는다는 점인데, 이를 One-Hot Encording을 통해 해결 할 수 있습니다.    
정수 치환으로 해결 할 수도 있지만 이럴 경우 성능이 저하되거나 예상치 못한 결과가 발생할 수 있습니다.

In [ ]:
#pandas에서는 get_dummies함수를 사용하면 쉽게 One-Hot Encording이 가능
cust1 = pd.get_dummies(cust_data['class'])

In [ ]:
# 문제
# cust_data 데이터 프레임 내 termination 변수를 원 핫 인코딩하여 범주형 데이터를 수치형 데이터로 변경하시오
cust2 =

In [ ]:
#columns를 사용해서 기존 테이블에 One-Hot Encording으로 치환된 변수 생성하기
cust_data_end=pd.get_dummies(cust_data, columns=['class'])
cust_data_end